In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from config import GptConfig
from model import GPT
from dataclasses import asdict
from pathlib import Path
torch.cuda.is_bf16_supported()

True

In [8]:
1215080/64

18985.625

In [3]:
# model = GPT(GptConfig(d_model=1024, n_layers=32, n_heads=8))
model = GPT(GptConfig(vocab_size=50304, d_model=1024, n_layers=32, n_heads=8, device='cuda'))

trainable_params = 0
non_trainable_params = 0

for params in model.parameters():
    if params.requires_grad:
        trainable_params+=params.numel()
    else:
        non_trainable_params+=params.numel()
    
print(f"""Trainable Parameters: {trainable_params/1e9:.3f}B
Non-Trainable Parameters: {non_trainable_params/1e9:.3f}B""")

Trainable Parameters: 0.506B
Non-Trainable Parameters: 0.000B


In [7]:
1024/12

85.33333333333333

In [ ]:
# load checkpoint
checkpoint_path = Path('checkpoints')
checkpoint_path = checkpoint_path.joinpath('ckpt_7950') 

model = GPT.from_pretrained(checkpoint_path,config=GptConfig(vocab_size=50304))
model.eval();

In [4]:
inputs = ['the best way to learn something is'] * 2
outputs = model.generate(text=inputs, temperature=.75, top_k=100)
for out in outputs:
    print(out)
    print("##"*30)

the best way to learn something isospels triggerseveryonegreg bruises recreationgregness simplrepresentMedicpmwikitersawan NinaLiquidained Lab dispersePink deitydocs assail heatLiquid Talksreverse foregoingirable assailante ColdKyprimedocs latConclusionra640 specialized colle Pale relies manage ul contingencypaid relies�ard
############################################################
the best way to learn something is bountpmwiki confronted Agility virtriction DEN ~/ companion baffilty year simpl ferry immunrepl participatingConclusion Agility Ltd Territory Carolyn DEN labeled execute Lossstice cuffAustral Fewogene loot1500Austral feeble apex loot feeble inability feebleKy Nina cuff messageschance vending PakTesting Domestic rag
############################################################


In [ ]:
# Save the model
model.save_pretrained(config=GptConfig(vocab_size=50304), save_directory='checkpoints', safe_serialization=False)

In [ ]:
# load other model
model.load_state_dict(torch.load('checkpoints\optimizer.pt')['model_state'])

In [ ]:
from huggingface_hub import upload_folder, upload_large_folder, create_repo
from huggingface_hub import login
from dotenv import load_dotenv
import os
load_dotenv()
login(os.getenv("HuggingFaceToken"))

create_repo(repo_id='Abdulvajid/gpt2-dataset', repo_type='dataset', exist_ok=True)
create_repo(repo_id='Abdulvajid/gpt2-from-scratch', repo_type='model', exist_ok=True)

# Upload everything in current dir
upload_folder(
    repo_id="Abdulvajid/gpt2-from-scratch",
    repo_type='model',
    folder_path="checkpoints",           # your project dir
    commit_message="Upload checkpoints",
    ignore_patterns=[".git", "__pycache__",".venv"]
)

# # Upload everything in current dir
# upload_folder(
#     repo_id="Abdulvajid/gpt2-dataset",
#     repo_type='checkpoints',
#     folder_path="pretrain_data",           # your project dir
#     # commit_message="Upload pretrain data",
#     ignore_patterns=[".git", "__pycache__",".venv"]
# )


In [ ]:
# Rotory Positional embedding
class RotoryPostionalEncoding(nn.Module):
    def __init__(self, conifg: GptConfig):
        super().__init__()
        

In [ ]:
import torch
d_model = 768
max_seq_len = 126

theta_range = torch.arange(0, d_model, 2)
print("sample of theta range: ", theta_range[:6])

In [ ]:
theta = 1.0 / (10000**(theta_range/d_model))
print(f"Theta sample : {theta[: 10]} and it's shape: {theta.shape}")

In [ ]:
positions = torch.arange(max_seq_len).unsqueeze(1) # [[0], [1]], (384, 1)
pos_thetas = positions * theta.unsqueeze(0) # (384 * 1) * (1, 384)

print(f"pos_theta's shape: {pos_thetas.shape}, sample:{pos_thetas[:20]}")


$$\cos m\theta_n$$
$$\sin m\theta_n$$

In [ ]:
# sin and cos of pos thetas
cos_pos_thetas = pos_thetas.cos()
sin_pos_thetas = pos_thetas.sin()

In [31]:
bcz = 10
seq = 50
d_model = 768

x = torch.randn(bcz, seq, d_model)


In [ ]:
even_x = x[..., 0::2]
odd_x = x[..., 1::2]
print(f"shape even_x:{even_x.shape}, odd_x:{odd_x.shape}")

shape even_x:torch.Size([10, 50, 384]), odd_x:torch.Size([10, 50, 384])


In [65]:
even_x.shape, cos_pos_thetas.shape

(torch.Size([10, 50, 384]), torch.Size([126, 384]))

In [41]:
even_rot = even_x * cos_pos_thetas[:50] - odd_x * sin_pos_thetas[:50]
odd_rot = odd_x * cos_pos_thetas[: 50] + even_x * sin_pos_thetas[: 50]

In [42]:
even_rot.shape, odd_rot.shape

(torch.Size([10, 50, 384]), torch.Size([10, 50, 384]))

In [48]:
even_rot[0], odd_rot[0]

(tensor([[-1.9249,  1.2316,  1.7821,  ..., -1.0023, -1.7774, -0.2267],
         [-0.1037, -0.3604, -0.0341,  ..., -0.1415, -0.2093, -0.4918],
         [-0.4292,  0.1906,  0.5339,  ...,  0.5126, -0.5034,  1.8498],
         ...,
         [ 1.0403,  1.0128,  0.4387,  ..., -0.8746, -1.1128, -0.9914],
         [ 0.7376,  1.0884, -0.9984,  ..., -0.4477,  0.2907, -2.1809],
         [-0.9391,  0.1999,  1.1407,  ..., -1.6766,  0.0159,  1.5735]]),
 tensor([[ 0.1735, -1.1102, -0.3941,  ...,  1.5987, -1.3216, -1.2584],
         [-1.4564,  1.4335,  0.2417,  ..., -1.2220, -0.8127,  1.0080],
         [-0.4650,  1.2145,  0.2853,  ..., -0.4937, -0.5569, -1.0821],
         ...,
         [-0.5121,  1.0466,  0.1559,  ..., -0.1942, -0.1316, -0.3880],
         [-1.9701,  0.8568, -0.3726,  ...,  0.4874, -0.6249, -0.1288],
         [ 0.2254, -0.7957,  0.9396,  ...,  0.5519, -0.3253,  1.1759]]))

In [ ]:
out = torch.stack((even_rot, odd_rot), dim=-1).flatten(2)

tensor([[[-1.9249e+00,  1.7347e-01,  1.2316e+00,  ..., -1.3216e+00,
          -2.2675e-01, -1.2584e+00],
         [-1.0367e-01, -1.4564e+00, -3.6043e-01,  ..., -8.1275e-01,
          -4.9180e-01,  1.0080e+00],
         [-4.2919e-01, -4.6497e-01,  1.9061e-01,  ..., -5.5688e-01,
           1.8498e+00, -1.0821e+00],
         ...,
         [ 1.0403e+00, -5.1208e-01,  1.0128e+00,  ..., -1.3161e-01,
          -9.9137e-01, -3.8796e-01],
         [ 7.3760e-01, -1.9701e+00,  1.0884e+00,  ..., -6.2494e-01,
          -2.1809e+00, -1.2880e-01],
         [-9.3905e-01,  2.2537e-01,  1.9991e-01,  ..., -3.2530e-01,
           1.5735e+00,  1.1759e+00]],

        [[ 5.4928e-01, -4.0640e-02,  6.8965e-01,  ..., -6.6069e-02,
           8.2892e-01, -4.2444e-01],
         [ 2.0306e+00,  2.5573e-01, -9.3429e-01,  ...,  5.7031e-01,
           2.2091e-01, -1.1832e+00],
         [-6.6490e-01,  8.7358e-01, -3.4540e-01,  ..., -2.1109e-01,
          -7.4313e-01, -1.4421e+00],
         ...,
         [ 9.3002e-01,  6

In [ ]:
import torch.nn as nn
from config import GptConfig
import config

class RotoryPositionalEncoding(nn.Module):
    def __init__(self, config: GptConfig):
        super().__init__()
        self.config = config
        self.precompute_thetas()
        
    def precompute_thetas(self):
        head_dim = self.config.d_model//self.config.n_heads # 128
        thetas = 1 / (torch.pow(10000, torch.arange(0, head_dim , 2)/head_dim))  # (64)
        token_positions = torch.arange(self.config.context_len) # (1024) $$\theta$$
        pos_thetas = torch.outer(token_positions, thetas) # (1024, 64) = ($$m_\theta$$) each pos head_dim have a theta
        self.register_buffer('cos_thetas', pos_thetas.cos().unsqueeze(0).unsqueeze(-2)) 
        self.register_buffer('sin_thetas', pos_thetas.sin().unsqueeze(0).unsqueeze(-2)) # (1, seq_len, 1, 64) for broadcasting
        
    def forward(self, x):
        bcz, seq_len, _, _ = x.size() # (bcz, seq, heads, head_dim)
        
        even_x = x[..., 0::2] # (bcz, seq, head, head_dim//2)
        odd_x = x[..., 1::2] # (bcz, seq, head, head_dim//2)
        
        # rotate
        even_rot = even_x * self.cos_thetas[..., :seq_len, :, :] - odd_x * self.sin_thetas[..., :seq_len, :, :]
        odd_rot = odd_x * self.cos_thetas[..., :seq_len, :, :] + even_x * self.sin_thetas[..., :seq_len, :, :]
        
        x_stacked = torch.stack((even_rot, odd_rot), dim=-1) # bcz,seq,d_model,2

        return x_stacked.flatten(-2)

In [38]:
rope = RotoryPositionalEncoding(config=GptConfig())

In [39]:
x = torch.randn(5, 10, 8, 64)

In [40]:
rope(x).shape

torch.Size([5, 10, 8, 64])

In [ ]:
x = torch.tensor([1, 2, 3],)
y = torch.tensor([0, 1, 2])

torch.outer(x, y)